In [1]:
# import sys
# import os
# import math
# import json
# import uuid
# import datetime
# import subprocess
# from pathlib import Path
# import numpy as np
# import pandas as pd
from tqdm import tqdm
# from models.td3 import TD3
# # from osim.env import ProstheticsEnv
# from environment.prosthetics_env_with_history import ProstheticsEnvWithHistory
# from environment.observations import prepare_model_observation, env_obs_history_to_model_obs
# from environment.actions import prepare_env_action, reset_frameskip
from environment.rewards import env_obs_to_custom_reward
from distributed.database import sample_timesteps, get_db_threadsafe
# from distributed.db_history_sampler import DatabaseHistorySampler
# from distributed.s3_checkpoints import load_s3_model_checkpoint, save_s3_model_checkpoint
# import torch
# import torch.utils.data




In [2]:
db = get_db_threadsafe()

In [3]:
episodes = db.timesteps.distinct('episode_uuid')
len(episodes), episodes[:5]

(6377,
 ['0006494d-88bb-4e37-ac6c-4012a3a597d5',
  '000da3a1-c927-4a95-80d7-4c0486a8813e',
  '001460fd-1dd3-49f1-b1d6-e1624c2ed501',
  '001ba46e-13f0-466f-8e1d-cb507b357ecd',
  '0025c6d4-fb76-4142-b5d4-bbb0b11f0f31'])

In [4]:
for eid in tqdm(episodes):
    steps = list(db.timesteps.find(filter={
        'episode_uuid': eid
    }))
    episode_reward = 0
    episode_custom_reward = 0
    for step in steps:
        episode_reward += step['reward']
        custom_reward = env_obs_to_custom_reward(step['obs'])
        sum_custom_reward = sum(custom_reward.values())
        episode_custom_reward += sum_custom_reward
        # print(step['_id'], step['reward'], episode_reward, sum_custom_reward, episode_custom_reward)
        db.timesteps.update_one(
            filter={
                '_id': step['_id']
            },
            update={
                '$set': {
                    'custom_rewards': custom_reward,
                    'custom_reward': sum_custom_reward,
                    'cum_reward': episode_reward,
                    'cum_custom_reward': episode_custom_reward
                }
            }
        )
# timesteps_without_custom_reward = db.timesteps.find(
#     filter={
#         '$or': [{
#             'episode_uuid': d['episode_uuid'],
#             'i_step': {
#                 '$in': [d['i_step']-h for h in range(1, 1+n_obs_history)],
#             },
#         } for d in docs],
#     }, 
#     projection={
#         'episode_uuid': True, 
#         'i_step': True, 
#         'obs': True
#     }
# )

100%|██████████| 6377/6377 [06:52<00:00, 15.44it/s]
